In [ ]:
directions = [5,15,20,25,35]
labels = ['extremeleft','left','middle','right','extremeright']
labeldict = dict(zip(directions,labels))
n_each = 15

In [ ]:
import random
import pandas as pd
import numpy as np
%pylab inline

In [ ]:
def create_random_design():
    dirs = []
    lastdir = None
    for _ in range(n_each):
        acceptable = False
        while not acceptable:
            d = directions[:]
            random.shuffle(d)
            acceptable = d[0]!=lastdir
        lastdir=d[-1]
        dirs+=d
    return dirs

In [ ]:
dirs = create_random_design()
" ".join([str(d) for d in dirs ])

## What are the carry-over effects?

In [ ]:
def get_transitions(dirs):
    # Get the first-order transition matrix
    trans = [ (dirs[i],dirs[i+1]) for i in range(len(dirs)-1) ]
    n = len(directions)
    transcount = np.zeros((n,n))
    for (d0,d1) in trans:
        d0i = directions.index(d0)
        d1i = directions.index(d1)
        transcount[d0i,d1i]+=1
    return transcount

In [ ]:
transcount = get_transitions(dirs)
transcount

## Batch-generate schedules and pick the best one
Generations of scholars will wonder why on Earth we do this, but hey...

In [ ]:
def badness(mat):
    # Tell us how bad this transition matrix is - how unequal are the transitional probability distributions?
    
    # Compute the average of each row
    mat = np.array(mat)
    n,_ = mat.shape
    # Restrict to the off-diagonal elements 
    rowvalues = [ vals[sel] for (vals,sel) in zip(mat,(1-np.eye(n)).astype(bool)) ]
    
    # Compute the deviation of each item from the average for that row
    dists = [ [ abs(r-np.mean(row)) for r in row ] for row in rowvalues ]
    
    return sum(np.array(dists).flatten())

In [ ]:
badness(transcount)

In [ ]:
#design

In [ ]:
N_DESIGNS = 100000
designs = []
for _ in range(N_DESIGNS):
    design = create_random_design()
    trans = get_transitions(design)
    howbad = badness(trans)
    designs.append((design,howbad))

In [ ]:
b = [ bad for _,bad in designs ]

In [ ]:
hist(b)

In [ ]:
best = min(b)
print(best)

In [ ]:
chosen,_ = designs[ b.index(best) ]

In [ ]:
#chosen

In [ ]:
trans = get_transitions(chosen)

In [ ]:
trans

## Output

In [ ]:
tab = pd.DataFrame({'direction':chosen})
tab['type']=[ labeldict[d] for d in chosen ]
tab['trial']=np.arange(len(chosen))+1
tab.to_csv('recognitionyesno_theOne.csv')

In [ ]:
tab

# Doing this the smart way (maybe)

In [ ]:
if False:
    #Highest number in square
    order_of_sq = int(input("Enter order of sq: "))

    #Number you want to start the square with
    top_left = int(input("Enter top left number: "))

    #Sets a placeholder for a variable called top_left_init
    top_left_init=0

    #Sets the initial value of top_left to a new variable because the code will change the value of top left later on 
    top_left_init += top_left

    #Initialize the value of count
    count = 0

    #Add 1 to the highest value in latin square to account for the range function (the ending number is always one less than the number you enter into the range function)
    for values in range (1,order_of_sq+1):

        #Prevents the program from adding too many characters to the line
        while count != order_of_sq:

            #Prints numbers with spaces after them in a horizontal manner
            print(top_left,sep=' ',end=' ')

            #Adds 1 to the top_left
            top_left += 1

            #Count is used to keep track of how many characters are in your line
            count+=1

            #Restarts the numbers in your line when you reach the highest number
            if top_left == order_of_sq+1:
                top_left = 1

        #Creates a new row
        print()
        count = 0

        #Calls the initial value of top_left and adds 1 to it before beginning the next row
        top_left_init += 1

        #Resets top_left_init to 1 if it reaches the highest number in the square
        if top_left_init == order_of_sq + 1:
            top_left_init = 1
            top_left = top_left_init
        else:
            top_left = top_left_init